# SETUP FOR SUPPORTING QUERIES OF IFCOWL GEOMETRIES CONVERTED INTO GEOSPARQL

Lecturer: **Kris McGlinn**, [www.krismcglinn.com](http://www.krismcglinn.com)  

Edited by Kris McGlinn, [ADAPT Centre](https://www.adaptcentre.ie), [Trinity College Dublin](https://www.tcd.ie), Ireland

## Introduction

This document aims to provide the reader with a step-by-step tutorial for converting ifcOWL IfcStandardWallCase geometries into GeoSPARQL and export them as ifcOWL with added GeoSPARQL and also BOT with GeoSPARQL. You will also learn how to write and execute some simple GeoSPARQL queries.
The objectives of the tutorial are as follows:  

* Show how to install the Jena code base for adding GeoSPARQL to a sample ifcOWL and BOT model.
* Demonstrate some simple queries over the resulting data.  

## CHOOSING AN APPROPRIATE IFCOWL FILE


The code you are going to use in this tutorial is still in development, and as such has only been tested on a small range of IFC Models which can be found [here](http://smartlab1.elis.ugent.be:8889/IFC-repo/) and which are named in Table 1. The previous site also provides a method for converting IFC step files into RDF, the source code of which is available here:

**Table 1:** Six ifcOWL files which can be found here [here](https://www.scss.tcd.ie/~mcglink/workshop/ldac2019/)

| File Name | RF Triple Count | Wall Count | Building Storey Count |
|:-|:-|:-|:-|
| smallhouse_saref.tll | 100246 | 6 | 1 |
| Duplex_A_20110907_optimized.ttl | 205141 | 56 | 4 |
| Simple3-storeytestbuilding.ttl | 222545 | 21 | 3 |
| 20170804_Musterhaus_MIT.ttl | 239899 | 24 | 3 |
| AC20-FZK Haus.ttl | 239899 | 13 | 2 |
| Barcelona_Pavilion.ttl | 1033905 | 35 | 2 |

Save your file as 'test_file.ttl' or someother name. In the rest of this tutorial we will use the name 'test_file.ttl' to indicate the ifcowl file.

## STEP 1: DOWNLOAD AND INSTALL THE CONVERTOR

You can access the code from github at the following address: [https://github.com/kmcglinn/fcOwl2IfcOwlGeo](https://github.com/kmcglinn/fcOwl2IfcOwlGeo)

Clone the project onto your computer and open it in your favorite IDE. I use [netbeans](https://netbeans.org/).

Alternatively, you can download the jar file and its dependencies from the following location: [https://www.scss.tcd.ie/~mcglink/workshop/ldac2019/](https://www.scss.tcd.ie/~mcglink/workshop/ldac2019/)  


## STEP 2: RUN THE CONVERTER AND EXPLORE THE OUTPUTS

Save your file into the folder where you generate your jar file (or have saved your jar file). Be sure to have the lib folder with all the dependencies in the same folder as your jar.

Run the convertor with the following command:

> java -jar IfcOwl2IfcOwlGeoloc-1.0-SNAPSHOT.jar “test_file.ttl” 

This should generate two files: a “test_file_geoloc.ttl” and “test_file_bot_geo.ttl” in a folder called “output”.

You can now open the outputs in any text editor. Look for the properties “hasGeometry” to find the resulting geometry, and “asWKT” the wkt literals associated with ifc and bot individuals.  Next you will set up a triplestore which is geoSPARQL enabled and begin writing some GeoSPARQL queries. 


## STEP 3 SETTING UP THE TRIPLESTORE AND GEOSPARQL 

Setting up a triplestore and SPARQL endpoint is straightforward. Several suites exist which support GeoSPARQL:

* [Stardog](http://stardog.com/) (free for non-commercial purposes)  
* [Parliament](http://parliament.semwebcentral.org/)  
* [Apache Jena and Fuseki](https://github.com/galbiston/geosparql-fuseki)  
* …  

For this tutorial, we are going to use Stardog. GeoSPARQL is only supported with the commercial license, so if your license has run out, Parliament is a good free option (and very easy to setup!). 

After you download stardog, extract it into a folder on your hard drive, following the instructions [here](https://www.stardog.com/docs/#_quick_start_guide)

For windows the following instructions are given:

First, tell Stardog where its home directory (where databases and other files will be stored) is:

> SET STARDOG_HOME=C:\data\stardog

Second, copy the stardog-license-key.bin into the right place:

> COPY /B stardog-license-key.bin %STARDOG_HOME%

Stardog won’t run without a valid Stardog-license-key.bin in STARDOG_HOME. You should have received one when you downloaded Stardog.

Third, optionally, place the bin folder of the Stardog install on your PATH so the stardog.bat and stardog-admin.bat scripts can be used regardless of current working directory

> SET PATH=%PATH%;C:\stardog-6.2.1\bin

You must also make sure you have a JDK installed. For Stardog 6.2.1 I used the latest JDK downloaded from [here](https://www.oracle.com/technetwork/java/javase/downloads/jdk8-downloads-2133151.html)

Before you start your server there are still a few more hoops to jump through. First you will need to create a stardog.properties file and save it into folder data/stardog. You can find the properties file [here](https://github.com/stardog-union/stardog-examples/blob/develop/config/stardog.properties)

You need to them comment out the line “spatial.use.jts=true” (remove the #).

Next you will need to download [JTS](http://central.maven.org/maven2/com/vividsolutions/jts-core/1.14.0/jts-core-1.14.0.jar) and save it into your stardog-6.2.1\server\ext 
(create the ext folder if it is not already there).

Now you can start Stardog!

For this tutorial we will use the web console provided by Stardog to check if our database has been successfully created (using the –web-console command). You also need to disable security so that it works with yasgui (see next section). Run the following command in your command line (if you have not set the path for Stardog, you can run this command from the bin folder in your Stardog download): 

> C:\stardog-admin.bat server start --web-console --disable-security

Once the database is running (you will see a command screen below if things went right) 


<img src="https://www.scss.tcd.ie/~mcglink/workshop/ldac2019/figures/img1.png" width="600" />

You will need to create a database with geospatial enabled. This is done using the following command in your command line (open a second command window if required). Here we call the db ifcgeo and we also load in our text data set “test_file_ ifc_geo.ttl”

> stardog-admin db create -o spatial.enabled=true -n ifcgeo c:/data/test.ttl

Now open your browser with the following url: [http://localhost:5820/](http://localhost:5820/)

You can check to make sure the db you created is there by clicking on “ifcgeo” (see below). Once you have clicked, check to see “spatial index” is set to “on”. 

<img src="https://www.scss.tcd.ie/~mcglink/workshop/ldac2019/figures/img2.png" width="600" />

<img src="https://www.scss.tcd.ie/~mcglink/workshop/ldac2019/figures/img3.png" width="600" />

With the triples loaded, you can now start playing with SPARQL queries. In the next tutorial, we look at this in more detail and you will create some GeoSPARQL queries over your geometry.

## STEP 4 RUNNING GEOSPARQL QUERIES OVER YOUR DATASET

The first place to test your SPARQL queries is through the Stardog web client. You can skip this part if you are familiar with SPARQL. 

Let’s start by querying all the data from your test file data with SPARQL SELECT. A simple SPARQL query looks something like this:

```sparql
SELECT ?subject
WHERE { ?subject  ?predicate  ?object }
```

so try something like the following to select all triples (you can generate this query automatically through the Stardog client interface) - 

```sparql
SELECT ?subject ?predicate ?object
WHERE {
  ?subject ?predicate ?object
}
LIMIT 25
```

You do not need to write subject, predicate, object and can simply write ?y,?x or ?y.

Now let us try to do a query using [YASGUI](http://yasgui.org/).

Upon opening yasgui in your browser, you will need to specify the endpoint for Stardog, which is [http://localhost:5820/ifcgeo/query](http://localhost:5820/ifcgeo/query)

<img src="https://www.scss.tcd.ie/~mcglink/workshop/ldac2019/figures/img4.png" width="600" />

Now let’s try your first query (above). This will return 10 wkt literals. You can now click on the Geo tab and see the visualization of the wkt literals in the yasgui interface. 

Now lets try a query using a geospatial function (change the coordinates below to the long and lat of the test file you are using (remember to reverse the long and lat):

```sparql
prefix geo:  <http://www.opengis.net/ont/geosparql#>
prefix geof: <http://www.opengis.net/def/function/geosparql/>

SELECT * WHERE{
 ?loc geo:hasGeometry ?geo .
 	?geo geof:nearby(49.100435 8.436539 0.01 <http://qudt.org/vocab/unit#Kilometer>).
#  	?geo geof:nearby(53.3509 -6.2611 1 <http://qudt.org/vocab/unit#Kilometer>).
 	?geo geo:asWKT ?wkt .
}
```

## STEP 5 WORKING WITH GEOSPATIAL DATA

Now let’s examine the use of some geospatial data. Go to (url will be provided on the day) and download the RDF file. This file is a subset of the Ordnance Survey Irelands geospatial data. They have over 3.5 million polygon geometries for buildings in Ireland. This data is not for public consumption, so please do not use this data beyond this tutorial.   

Upload this data into stardog following the previous procedure. Now use yasgui to query the data. Try the previous geospatial queries over this new data set, in particular the nearby function. 

Finally, as a challenge, see if you can edit an IFC models Longitude and Latitude to something close to the geospatial data, then convert the IFC geometry to WKT using the given method. How would you go about aligning an IFC geometry with geospatial data provided in the same form as the data provided by OSi?  
